# Sales Prediction as a Regression Problem

We can consider two main approaches for this unit-sales prediction task

1. Treat the problem as a time-series forecasting problem, i.e. for each item_number, consider past sales as a time-series and predict for a given duration
2. Treat the problem as a multi-variate regression problem, i.e. consider the input and target as non-time-series but treat date as a separate input.

I opted for the second approach for the following reasons

1. Developing a separate model for individual items to predict their future sales seems impractical. Especially when adding a new item, you end up with no data to train a model.
2. Treating this as a multivariate regression problem allows me to use a wider range of possible models.
3. Using a proper model, seasonal variations etc. may be implicitly captured, but deseasonalising data as preprocessing is preferred to make it easier for the model to learn the patterns

## Feature Engineering and Preprocessing

Two main types of data: Numerical and Categorical. Numerical data can be fed as is to the model with some normalisation/standardisation. Categorical data must be one-hot-encoded / bitmap indexed to allow identifying decision boundaries without arbitrary assignment of numerical values to the categories. However, for cases such as 'store-nbr', 'item.class', and 'item_nbr' where the cardinality is considerably large, onehot encoding may add to curse of dimensionality, so I will keep it as numerical for now.

Numerical Data will be standardised (to allow for out-of-sample integration which may not be possible with normalisation).

## Handling out-of-sample data points in the test-data

For now, my approach only considers predicting the data for values contained within the training set, and I have used a train-test-split to try and validate my models for this. Unfortunately there is a few data points not contained within the trainingset but is present in the test data provided by the kaggle competition. Here is my strategy to deal with them if more time is provided.

1. De-identify the data: Variables such as the item_nbr and store_nbr may be subject to new additions (new product introductions, opening new stores e.g.). Therefore, I would attempt to represent a store by other related variables and create a surrogate store-profile to replace the store_nbr (store location, store type etc.). Similarly, I would represent a product with associated other variables and not the item_nbr.
2. Replace Locale information with geographic information: Some holiday information is not contained within the training set, e.g. holidays in Puyo, Pastaz (Ecuador) but is contained in the test set. I would rely on doing a web-crawl to replace such locales with the 'geographically nearest' locale contained in the training set.

### References

I first read the problem description and went through the existing solutions (the code kernels on kaggle). I have referred to some of them to load the large files onto my local computer's ram as I had trouble with them. But the data workflow I have written on my own as I did not agree with some of the processing steps in them.

In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
items = pd.read_csv("./items.csv")
holiday_events = pd.read_csv("./holidays_events.csv", parse_dates=['date'])
stores = pd.read_csv("./stores.csv")
oil = pd.read_csv("./oil.csv", parse_dates=['date'])
transactions = pd.read_csv("./transactions.csv", parse_dates=['date'])
train = pd.read_csv("./train.csv", nrows=1000000  , parse_dates=['date'])
# train_large = pd.read_csv('./train.csv', skiprows = 115000000, names = train.columns, parse_dates = ['date'])

In [3]:
test = pd.read_csv('./test.csv', parse_dates=['date'])

In [4]:

def merge_with_metadata(source, oil, stores, items, holidays):
    source.onpromotion = source.onpromotion.fillna(False)
    oil.dcoilwtico = oil.dcoilwtico.bfill()
    source = pd.merge(source, oil, how='left', left_on='date', right_on='date')
    source = pd.merge(source, items, how='left', left_on='item_nbr', right_on='item_nbr')
    source = pd.merge(source, holidays, how = 'left', left_on='date', right_on='date')
    source = pd.merge(source, stores, how = 'left', left_on='store_nbr', right_on='store_nbr')
    source['date'] = source['date'].map(dt.datetime.toordinal)
    source.type_x = source.type_x.fillna('Not a Holiday')
    source.locale = source.locale.fillna('NA')
    source.locale_name = source.locale_name.fillna('NA')
    source.transferred = source.transferred.astype(str).fillna('NA')
    source.onpromotion = source.onpromotion.astype(float)
    source.drop(columns = ['id'], inplace=True)
    numeric_columns = source.drop(columns = [ 'cluster'])._get_numeric_data().columns
    categorical_columns = (np.setdiff1d(source.columns , numeric_columns))
    num_df = source[numeric_columns].drop(columns = ['date', 'unit_sales'])
    cat_df = source[categorical_columns].astype(str)

    date_df = source['date']
    target_df = source['unit_sales']

    return num_df, cat_df, date_df, target_df

In [5]:
numerical_values, categorical_values, date_values, target_values = merge_with_metadata(train, oil, stores, items, holiday_events)
print(numerical_values.columns)
print(categorical_values.columns)

Index(['store_nbr', 'item_nbr', 'onpromotion', 'dcoilwtico', 'class',
       'perishable'],
      dtype='object')
Index(['city', 'cluster', 'description', 'family', 'locale', 'locale_name',
       'state', 'transferred', 'type_x', 'type_y'],
      dtype='object')


In [6]:
categorical_values.head()

,city,cluster,description,family,locale,locale_name,state,transferred,type_x,type_y
0,Salinas,1,Primer dia del ano,BREAD/BAKERY,National,Ecuador,Santa Elena,False,Holiday,D
1,Salinas,1,Primer dia del ano,GROCERY I,National,Ecuador,Santa Elena,False,Holiday,D
2,Salinas,1,Primer dia del ano,GROCERY I,National,Ecuador,Santa Elena,False,Holiday,D
3,Salinas,1,Primer dia del ano,GROCERY I,National,Ecuador,Santa Elena,False,Holiday,D
4,Salinas,1,Primer dia del ano,DELI,National,Ecuador,Santa Elena,False,Holiday,D


In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
num_df = pd.DataFrame(StandardScaler().fit_transform(numerical_values.values), columns = numerical_values.columns)


In [9]:
categorical_values = pd.get_dummies(categorical_values, categorical_values.columns)

In [10]:

categorical_values.head()

,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_El Carmen,city_Esmeraldas,city_Guaranda,city_Guayaquil,city_Ibarra,...,transferred_False,transferred_nan,type_x_Holiday,type_x_Not a Holiday,type_x_Work Day,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0


In [11]:
train_merged = pd.concat([num_df, categorical_values.astype(float), date_values], axis=1)

In [12]:
train_merged.head()

,store_nbr,item_nbr,onpromotion,dcoilwtico,class,perishable,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,...,transferred_nan,type_x_Holiday,type_x_Not a Holiday,type_x_Work Day,type_y_A,type_y_B,type_y_C,type_y_D,type_y_E,date
0,-0.081546,-1.693226,0.0,-1.010501,0.753458,1.881282,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,734869
1,-0.081546,-1.686936,0.0,-1.010501,-0.829665,-0.531552,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,734869
2,-0.081546,-1.686932,0.0,-1.010501,-0.829665,-0.531552,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,734869
3,-0.081546,-1.678682,0.0,-1.010501,-0.843910,-0.531552,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,734869
4,-0.081546,-1.676632,0.0,-1.010501,0.688880,1.881282,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,734869


In [13]:
#X_tr = (train_merged[['date', 'onpromotion', 'item_nbr','locale', 'locale_name', 'transferred', 'store_nbr', 'dcoilwtico', 'family', 'class', 'perishable', 'cluster']]).values
X_tr = train_merged.values

In [14]:
from sklearn.preprocessing import StandardScaler

# X_tr = StandardScaler().fit_transform(X_tr)

In [15]:
X_tr = np.nan_to_num(X_tr).astype(float)
print(np.isnan(X_tr).any())

False


In [16]:
Y_tr = target_values.values

In [17]:
'''validation on training set'''
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(X_tr, Y_tr, test_size=0.2)

In [18]:
print(train_x.shape, test_x.shape)

(800000, 97) (200000, 97)


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

In [20]:
rf = RandomForestRegressor()

In [ ]:
rf.fit(train_x, train_y)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
# print(train_x)
prediction = rf.predict(test_x)
#print(train_merged.head())

In [ ]:
print(f'rmse = {np.sqrt(mean_squared_error(test_y, prediction))}')
print(f'mean_absolute_error = {mean_absolute_error(test_y, prediction)}')

In [ ]:
print(prediction, test_y)

In [ ]:
test_merged = merge_with_metadata(test, oil, stores, items, holiday_events)
